### Algoritmo de Busqueda Simulated Anneling - TSP

#### Solución al problema del vendedor ambulante utilizando el módulo simanneal

El problema de optimización discreta por excelencia es el problema del viajante (TSP por sus siglas en ingles):

| Dada una lista de ubicaciones, ¿cuál es la ruta más corta posible que llega a cada ubicación y regresa a la ciudad de partida?

Para ponerlo en términos de nuestro marco de recocido simulado:

* El **estado** es una lista ordenada de lugares para visitar.
* El **movimiento** baraja dos ciudades en la lista.
* La **energía** de un estado dado es la distancia recorrida

#### Módulo de Python para Simulated Anneling

Este módulo realiza una optimización de recocido simulado para encontrar el estado óptimo de un sistema. Está inspirado en el proceso metalúrgico de recocido mediante el cual los metales deben enfriarse en un horario regular para asentarse en su estado de energía más bajo.

El recocido simulado se utiliza para encontrar una solución cercana a la óptima entre un conjunto extremadamente grande (pero finito) de soluciones potenciales. Es particularmente útil para problemas de optimización combinatoria definidos por funciones objetivas complejas que dependen de datos externos.

El proceso implica:

* Mover o alterar aleatoriamente el estado
* Evaluar la energía del nuevo estado usando una función objetivo.
* Comparar la energía con el estado anterior y decida si acepta la nueva solución o la rechaza en función de la temperatura actual .
* Repitir hasta que haya convergido en una respuesta aceptable.

Para que un tour sea aceptado, debe cumplir uno de dos requisitos:

* El movimiento provoca una disminución en la energía del estado (es decir, una mejora en la función objetivo)
* El movimiento aumenta la energía del estado (es decir, una solución ligeramente peor) pero está dentro de los límites de la temperatura. La temperatura disminuye exponencialmente a medida que avanza el algoritmo. De esta manera, evitamos quedar atrapados por los mínimos locales al principio del proceso, pero comenzamos a afinar una solución viable al final.

#### 1. Instalación de libreria simanneal

Instalar la libreria **simanneal**

pip instalar simanneal   # desde pypi

#### 2. Importación de librerías necesarias

In [11]:
from __future__ import print_function
import math
import random
from collections import defaultdict
from simanneal import Annealer

#### 3. Tour de ciudades

Definimos la lista de ciudades a visitar.

In [12]:
if __name__ == '__main__':

    # latitud y longitud de las veinte ciudades más grandes de EE. UU.
    cities = {
        'New York City': (40.72, 74.00),
        'Los Angeles': (34.05, 118.25),
        'Chicago': (41.88, 87.63),
        'Houston': (29.77, 95.38),
        'Phoenix': (33.45, 112.07),
        'Philadelphia': (39.95, 75.17),
        'San Antonio': (29.53, 98.47),
        'Dallas': (32.78, 96.80),
        'San Diego': (32.78, 117.15),
        'San Jose': (37.30, 121.87),
        'Detroit': (42.33, 83.05),
        'San Francisco': (37.78, 122.42),
        'Jacksonville': (30.32, 81.70),
        'Indianapolis': (39.78, 86.15),
        'Austin': (30.27, 97.77),
        'Columbus': (39.98, 82.98),
        'Fort Worth': (32.75, 97.33),
        'Charlotte': (35.23, 80.85),
        'Memphis': (35.12, 89.97),
        'Baltimore': (39.28, 76.62)
    }

Es necesario calcular la distancia entre dos ciudades a partir de sus coordenadas.  Para ello, definimos la funcion **distance**

In [13]:
def distance(a, b):
    """Calcula la distancia entre dos coordenadas de latitud y longitud."""
    R = 3963  # radio de la Tierra (millas)
    lat1, lon1 = math.radians(a[0]), math.radians(a[1])
    lat2, lon2 = math.radians(b[0]), math.radians(b[1])
    return math.acos(math.sin(lat1) * math.sin(lat2) +
                     math.cos(lat1) * math.cos(lat2) * math.cos(lon1 - lon2)) * R

Se crea una matriz de distancias para cada item de la lista de ciudades posibles a visitar

In [14]:
# Se crea una matriz de distancia
distance_matrix = defaultdict(dict)
for ka, va in cities.items():
    for kb, vb in cities.items():
        distance_matrix[ka][kb] = 0.0 if kb == ka else distance(va, vb)

#### 3. Definición del problema

Para definir nuestro problema, creamos una clase que hereda de simanneal.Annealer

Dentro de esa clase, definimos dos métodos obligatorios. 
* Primero: definimos el **movimiento**
* Segundo: definimos cómo se calcula **la energía** (también conocida como función objetivo)

Tener en cuenta que ambos métodos tienen acceso al self.state que rastrea el estado actual del proceso.

In [15]:
class TravellingSalesmanProblem(Annealer):

    """Prueba del Algoritmo Simulated Anneling con un problema de vendedor ambulante.
    """

    # pass extra data (the distance matrix) into the constructor
    def __init__(self, state, distance_matrix):
        self.distance_matrix = distance_matrix
        super(TravellingSalesmanProblem, self).__init__(state)  # important!

    def move(self):
        """Intercambia dos ciudades en la ruta"""
        # no se gana en eficiencia, solo se prueba eloncepto
        # demuestra la devolución de la energía delta (opcional)
        initial_energy = self.energy()

        a = random.randint(0, len(self.state) - 1)
        b = random.randint(0, len(self.state) - 1)
        self.state[a], self.state[b] = self.state[b], self.state[a]

        return self.energy() - initial_energy

    def energy(self):
        """Calcula la longitud de la ruta"""
        e = 0
        for i in range(len(self.state)):
            e += self.distance_matrix[self.state[i-1]][self.state[i]]
        return e

In [40]:
# Se actualiza el estado inicial, se elige un itinerario ordenado al azar
init_state = list(cities)
random.shuffle(init_state)

El algoritmo de recocido simulado requiere que realicemos un seguimiento de los estados (actual, anterior, mejor), lo que significa que debemos copiar con self.state frecuencia.

Copiar un objeto en Python no siempre es sencillo o eficaz. La biblioteca estándar proporciona un copy.deepcopy()método para copiar objetos de Python arbitrarios, pero es muy costoso. Ciertos objetos se pueden copiar por medios más eficientes: las listas se pueden dividir y los diccionarios pueden usar su propio método .copy, etc.

Para facilitar la flexibilidad, se puede especificar el atributo copy_strategy que define uno de estos tipos de copy:

* deepcopy: usos copy.deepcopy(object)
* slice: usos object[:]
* method: usos object.copy()

Ahora, con nuestro problema especificado, podemos construir una instancia de **TravellingSalesmanProblem** y proporcionarle un estado inicial.

In [41]:
tsp = TravellingSalesmanProblem(init_state, distance_matrix)
tsp.set_schedule(tsp.auto(minutes=0.5))

# dado que nuestro estado es solo una lista, el segmento (slice) es la forma más rápida de copiar
tsp.copy_strategy = "slice"
state, e = tsp.anneal()

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.50000       6898.57     5.70%     0.00%     0:00:02    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.50000       6801.84     5.07%     0.00%     0:00:13     0:00:00

In [44]:
#Se puede especificar parametros
Tmax  =  25000.0   # Temperatura máxima (inicial) 
Tmin  =  2.5       # Temperatura máxima (final) 
tsp = TravellingSalesmanProblem(init_state, distance_matrix)
tsp.Tmax
tsp.Tmin
tsp.set_schedule(tsp.auto(minutes=0.3))
# dado que nuestro estado es solo una lista, el segmento (slice) es la forma más rápida de copiar
tsp.copy_strategy = "slice"
state, e = tsp.anneal()

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.87000       7046.86     5.00%     0.00%     0:00:03    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.87000       6801.84     5.05%     0.00%     0:00:09     0:00:00

In [47]:
#Se puede cambiar la ciudad desde la que se desea iniciar el tour
while state[0] != 'New York City':
        state = state[1:] + state[:1]  # rotar NYC para empezar

In [46]:
print()
print("%i millas de la ruta:" % e)
print(" ➞  ".join(state))


6801 millas de la ruta:
New York City ➞  Detroit ➞  Columbus ➞  Indianapolis ➞  Chicago ➞  San Francisco ➞  San Jose ➞  Los Angeles ➞  San Diego ➞  Phoenix ➞  San Antonio ➞  Austin ➞  Houston ➞  Fort Worth ➞  Dallas ➞  Memphis ➞  Jacksonville ➞  Charlotte ➞  Baltimore ➞  Philadelphia
